In [1]:
!pip3 install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import sys
sys.path.append('/home/jovyan/ChestXray-14')

In [3]:
import pandas as pd
import tensorflow as tf
from modules.utils import get_dataset
from modules.evaluate import Evaluate
from modules.dataset import Dataset

2023-02-22 09:30:35.081675: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
import os
CURRENT_PATH = os.path.abspath("")
CURRENT_PATH

'/home/jovyan/ChestXray-14/run_specific_fold'

In [5]:
RESULT_EVALUATE_PATH = os.path.join(CURRENT_PATH, "results", "evaluate", "facal_loss", "EfficientNetB0_None")
RESULT_EVALUATE_PATH

'/home/jovyan/ChestXray-14/run_specific_fold/results/evaluate/facal_loss/EfficientNetB0_None'

In [6]:
from pathlib import Path
Path(RESULT_EVALUATE_PATH).mkdir(parents=True, exist_ok=True)

In [7]:
dataset = Dataset()
fold_num = 3 # use values [1-5]
train_dataset, test_dataset = dataset.get_kfold(fold_num, sample=False)

2023-02-22 09:30:37.151319: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 09:30:38.942930: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31725 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:17:00.0, compute capability: 8.0


In [8]:
MODEL_PATH = f'{CURRENT_PATH}/results/models/facal_loss/EfficientNetB0_None_fold_3.h5'
MODEL_PATH

'/home/jovyan/ChestXray-14/run_specific_fold/results/models/facal_loss/EfficientNetB0_None_fold_3.h5'

In [9]:
best_model = Evaluate(MODEL_PATH)
best_model.get_best_threshold(
    test_dataset=test_dataset,
    save_best_thresholds=f"{RESULT_EVALUATE_PATH}/best_thresholds.csv",
    save_200_thresholds=f"{RESULT_EVALUATE_PATH}/f1_per_thresholds.csv"
)

with best_model:
    f1_each_class = best_model.get_f1_scores(test_dataset)
    df = pd.DataFrame(f1_each_class)
    df.to_csv(f"{RESULT_EVALUATE_PATH}/f1_scores.csv", index=False)

    precision_each_class = best_model.get_precision_scores(test_dataset, new_calculate=False)
    pd.DataFrame(precision_each_class)\
        .to_csv(f"{RESULT_EVALUATE_PATH}/precision.csv", index=False)

    recall_each_class = best_model.get_recall_scores(test_dataset, new_calculate=False)
    pd.DataFrame(recall_each_class)\
        .to_csv(f"{RESULT_EVALUATE_PATH}/recall.csv", index=False)

    print(df)

2023-02-22 09:31:02.388957: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-22 09:31:03.590925: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-22 09:31:03.593033: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-22 09:31:03.593133: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-02-22 09:31:03.594513: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-22 09:31:03.594728: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


      7/Unknown - 9s 27ms/step

2023-02-22 09:31:08.005921: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1402/1402 [==============================] - 53s 31ms/step
['No Finding', 'Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Mass', 'Hernia']
/home/jovyan/ChestXray-14/run_specific_fold/results/evaluate/facal_loss/EfficientNetB0_None/best_thresholds.csv was success!
/home/jovyan/ChestXray-14/run_specific_fold/results/evaluate/facal_loss/EfficientNetB0_None/f1_per_thresholds.csv was success!
Doing ...!
1402/1402 [==============================] - 41s 29ms/step
   No Finding  Atelectasis  Consolidation  Infiltration  Pneumothorax  \
0    0.741261     0.338441       0.199394      0.379445      0.282472   

      Edema  Emphysema  Fibrosis  Effusion  Pneumonia  Pleural_Thickening  \
0  0.209749   0.146032  0.078431  0.457168   0.063008            0.158212   

   Cardiomegaly    Nodule      Mass    Hernia  
0      0.336765  0.160393  0.221185  0.016667  
Done!
